In [8]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Masking, LSTM, Bidirectional, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [9]:
landmarks = pd.read_csv(r"C:\Users\andjelija.jovanovic\Desktop\movement project\data\landmarks.csv")
angles    = pd.read_csv(r"C:\Users\andjelija.jovanovic\Desktop\movement project\data\angles.csv")
labels    = pd.read_csv(r"C:\Users\andjelija.jovanovic\Desktop\movement project\data\labels.csv")

print("LANDMARKS:", landmarks.shape)
print("ANGLES:", angles.shape)
print("LABELS:", labels.shape)

landmarks.head()


LANDMARKS: (83922, 101)
ANGLES: (83922, 9)
LABELS: (448, 2)


,vid_id,frame_order,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,z_left_heel,x_right_heel,y_right_heel,z_right_heel,x_left_foot_index,y_left_foot_index,z_left_foot_index,x_right_foot_index,y_right_foot_index,z_right_foot_index
0,0,0,-0.645851,-59.992630,-80.98500,0.560464,-62.555250,-76.384210,1.362609,-62.543415,...,42.493310,-4.885307,67.512770,40.333897,5.356711,73.934240,11.780330,-5.852993,73.78203,9.016774
1,0,1,-0.290473,-61.069310,-78.47870,0.881309,-63.674810,-73.719315,1.639633,-63.648945,...,48.487360,-4.753275,64.969570,45.439384,5.492989,73.177270,18.108229,-6.038326,72.70349,14.222010
2,0,2,-0.378156,-61.102000,-86.33219,0.968603,-63.431263,-81.922356,1.788657,-63.423435,...,49.983517,-4.517086,64.510980,48.996880,5.433758,72.199036,19.192911,-5.513490,71.79309,17.322145
3,0,3,-0.004211,-61.846817,-98.94910,1.419466,-64.424550,-94.673550,2.102673,-64.361015,...,53.762500,-4.674540,64.720245,53.581780,5.768750,72.696290,23.325266,-5.238461,72.11217,21.887375
4,0,4,0.215262,-59.717796,-96.07627,1.495876,-62.196190,-91.907270,2.157559,-62.149612,...,53.409090,-4.098778,62.490230,52.845634,5.633003,70.438194,23.657516,-5.467475,70.08317,22.496626


In [10]:

data = landmarks.merge(
    angles,
    on=["vid_id", "frame_order"],
    how="inner"
)


data = data.merge(
    labels,
    on="vid_id",
    how="inner"
)

print("DATA shape:", data.shape)
data.head()


DATA shape: (83922, 109)


,vid_id,frame_order,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,y_right_foot_index,z_right_foot_index,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder,class
0,0,0,-0.645851,-59.992630,-80.98500,0.560464,-62.555250,-76.384210,1.362609,-62.543415,...,73.78203,9.016774,16.926802,7.667874,18.982162,112.747505,112.62553,112.099300,101.05565,jumping_jack
1,0,1,-0.290473,-61.069310,-78.47870,0.881309,-63.674810,-73.719315,1.639633,-63.648945,...,72.70349,14.222010,14.199318,8.954973,18.966124,109.707190,109.76263,110.645454,102.00027,jumping_jack
2,0,2,-0.378156,-61.102000,-86.33219,0.968603,-63.431263,-81.922356,1.788657,-63.423435,...,71.79309,17.322145,18.065800,10.315741,17.527954,114.562100,112.08965,113.340350,104.09502,jumping_jack
3,0,3,-0.004211,-61.846817,-98.94910,1.419466,-64.424550,-94.673550,2.102673,-64.361015,...,72.11217,21.887375,23.270214,17.336140,17.195545,117.674810,115.43172,114.634530,107.38297,jumping_jack
4,0,4,0.215262,-59.717796,-96.07627,1.495876,-62.196190,-91.907270,2.157559,-62.149612,...,70.08317,22.496626,22.831680,13.822096,17.355429,117.536720,117.96766,112.306390,98.39078,jumping_jack


In [11]:
def normalize_landmarks_df(df):
    df = df.copy()
    
    req_cols = ["x_left_hip", "y_left_hip", "z_left_hip",
                "x_right_hip", "y_right_hip", "z_right_hip"]
    if not all(col in df.columns for col in req_cols):
        print("Nedostaju hip kolone, preskačem normalizaciju.")
        return df
    
    df["mid_hip_x"] = (df["x_left_hip"] + df["x_right_hip"]) / 2
    df["mid_hip_y"] = (df["y_left_hip"] + df["y_right_hip"]) / 2
    df["mid_hip_z"] = (df["z_left_hip"] + df["z_right_hip"]) / 2

    coord_cols = [c for c in df.columns if c.startswith(("x_", "y_", "z_"))]

    for c in coord_cols:
        if c.startswith("x_"):
            df[c] = df[c] - df["mid_hip_x"]
        elif c.startswith("y_"):
            df[c] = df[c] - df["mid_hip_y"]
        elif c.startswith("z_"):
            df[c] = df[c] - df["mid_hip_z"]

    df = df.drop(columns=["mid_hip_x", "mid_hip_y", "mid_hip_z"])
    return df

data = data.groupby("vid_id", group_keys=False).apply(normalize_landmarks_df)
data.head()


C:\Users\andjelija.jovanovic\AppData\Local\Temp\ipykernel_27364\1354448417.py:27: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  data = data.groupby("vid_id", group_keys=False).apply(normalize_landmarks_df)


,vid_id,frame_order,x_nose,y_nose,z_nose,x_left_eye_inner,y_left_eye_inner,z_left_eye_inner,x_left_eye,y_left_eye,...,y_right_foot_index,z_right_foot_index,right_elbow_right_shoulder_right_hip,left_elbow_left_shoulder_left_hip,right_knee_mid_hip_left_knee,right_hip_right_knee_right_ankle,left_hip_left_knee_left_ankle,right_wrist_right_elbow_right_shoulder,left_wrist_left_elbow_left_shoulder,class
0,0,0,-0.645851,-59.992637,-80.98500,0.560464,-62.555257,-76.384210,1.362609,-62.543422,...,73.782023,9.016774,16.926802,7.667874,18.982162,112.747505,112.62553,112.099300,101.05565,jumping_jack
1,0,1,-0.290480,-61.069310,-78.47870,0.881303,-63.674810,-73.719315,1.639626,-63.648945,...,72.703490,14.222010,14.199318,8.954973,18.966124,109.707190,109.76263,110.645454,102.00027,jumping_jack
2,0,2,-0.378149,-61.102000,-86.33219,0.968610,-63.431263,-81.922356,1.788664,-63.423435,...,71.793090,17.322145,18.065800,10.315741,17.527954,114.562100,112.08965,113.340350,104.09502,jumping_jack
3,0,3,-0.004218,-61.846824,-98.94910,1.419460,-64.424557,-94.673550,2.102666,-64.361022,...,72.112163,21.887375,23.270214,17.336140,17.195545,117.674810,115.43172,114.634530,107.38297,jumping_jack
4,0,4,0.215262,-59.717803,-96.07627,1.495876,-62.196197,-91.907270,2.157559,-62.149619,...,70.083163,22.496626,22.831680,13.822096,17.355429,117.536720,117.96766,112.306390,98.39078,jumping_jack


In [12]:
feature_cols = [
    c for c in data.columns
    if c not in ["vid_id", "frame_order", "class"]
]

len(feature_cols), feature_cols[:10]


(106,
 ['x_nose',
  'y_nose',
  'z_nose',
  'x_left_eye_inner',
  'y_left_eye_inner',
  'z_left_eye_inner',
  'x_left_eye',
  'y_left_eye',
  'z_left_eye',
  'x_left_eye_outer'])

In [ ]:
videos = []
video_labels = []

for vid, group in data.groupby("vid_id"):
    group = group.sort_values("frame_order")
    
    seq = group[feature_cols].values.astype("float32")   
    label = group["class"].iloc[0]
    
    videos.append(seq)
    video_labels.append(label)

len(videos), len(video_labels)


(448, 448)

In [14]:
videos[0].shape, video_labels[0]


((265, 106), 'jumping_jack')

In [15]:
X_train_list, X_temp_list, y_train, y_temp = train_test_split(
    videos,
    video_labels,
    test_size=0.3,
    random_state=42,
    stratify=video_labels
)

X_val_list, X_test_list, y_val, y_test = train_test_split(
    X_temp_list,
    y_temp,
    test_size=0.5,
    random_state=42,
    stratify=y_temp
)

len(X_train_list), len(X_val_list), len(X_test_list)


(313, 67, 68)

In [16]:
le = LabelEncoder()

y_train_enc = to_categorical(le.fit_transform(y_train))
y_val_enc   = to_categorical(le.transform(y_val))
y_test_enc  = to_categorical(le.transform(y_test))

num_classes = y_train_enc.shape[1]
num_classes, le.classes_


(5,
 array(['jumping_jack', 'pull_up', 'push_up', 'situp', 'squat'],
       dtype='<U12'))

In [17]:
max_len = max(seq.shape[0] for seq in X_train_list)
num_features = X_train_list[0].shape[1]
max_len, num_features


(301, 106)

In [18]:
def pad_list(seqs):
    return pad_sequences(
        seqs,
        maxlen=max_len,
        dtype="float32",
        padding="post",
        truncating="post",
        value=0.0
    )

X_train = pad_list(X_train_list)
X_val   = pad_list(X_val_list)
X_test  = pad_list(X_test_list)

X_train.shape, X_val.shape, X_test.shape


((313, 301, 106), (67, 301, 106), (68, 301, 106))

In [19]:
model = Sequential([
    Input(shape=(max_len, num_features)),
    Masking(mask_value=0.0),
    Bidirectional(LSTM(64, return_sequences=True)),
    Dropout(0.3),
    Bidirectional(LSTM(32)),
    Dropout(0.3),
    Dense(64, activation="relu"),
    Dropout(0.3),
    Dense(num_classes, activation="softmax")
])

model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

model.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking (Masking)               │ (None, 301, 106)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 301, 128)       │        87,552 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 301, 128)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 64)             │        41,216 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 5)              │           325 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 133,253 (520.52 KB)

 Trainable params: 133,253 (520.52 KB)

 Non-trainable params: 0 (0.00 B)

In [20]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

checkpoint_path = "best_lstm_angles.keras"

callbacks = [
    EarlyStopping(
        monitor="val_loss",
        patience=5,
        restore_best_weights=True
    ),
    ModelCheckpoint(
        filepath=checkpoint_path,
        monitor="val_loss",
        save_best_only=True
    )
]

history = model.fit(
    X_train, y_train_enc,
    epochs=30,
    batch_size=8,
    validation_data=(X_val, y_val_enc),
    callbacks=callbacks,
    verbose=1
)


Epoch 1/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 12s 148ms/step - accuracy: 0.4569 - loss: 1.3777 - val_accuracy: 0.6716 - val_loss: 1.1075
Epoch 2/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step - accuracy: 0.6869 - loss: 0.9778 - val_accuracy: 0.7164 - val_loss: 0.8632
Epoch 3/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.7732 - loss: 0.7087 - val_accuracy: 0.7164 - val_loss: 0.6922
Epoch 4/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 131ms/step - accuracy: 0.8115 - loss: 0.5047 - val_accuracy: 0.7463 - val_loss: 0.7731
Epoch 5/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 128ms/step - accuracy: 0.8754 - loss: 0.4106 - val_accuracy: 0.7910 - val_loss: 0.5629
Epoch 6/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 122ms/step - accuracy: 0.8339 - loss: 0.4405 - val_accuracy: 0.7910 - val_loss: 0.6299
Epoch 7/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 120ms/step - accuracy: 0.8179 - loss: 0.4383 - val_accuracy: 0.8209 - val_loss: 0.5855
Epoch 8/30
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 129ms/step - accuracy: 0.8946 - loss: 0.3049 - val_accuracy: 0

In [21]:
test_loss, test_acc = model.evaluate(X_test, y_test_enc, verbose=1)
print(f"Test accuracy: {test_acc:.3f}, test loss: {test_loss:.3f}")


3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.7941 - loss: 0.6117
Test accuracy: 0.794, test loss: 0.612


In [22]:
y_test_pred_probs = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred_probs, axis=1)
y_test_true = np.argmax(y_test_enc, axis=1)

cm = confusion_matrix(y_test_true, y_test_pred)
print("Confusion matrix:\n", cm)

print("\nKlasifikacioni izveštaj:\n")
print(classification_report(y_test_true, y_test_pred, target_names=le.classes_))


3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 432ms/step
Confusion matrix:
 [[13  2  0  0  1]
 [ 0 14  1  0  0]
 [ 0  0 15  0  0]
 [ 0  2  0 10  0]
 [ 2  4  0  2  2]]

Klasifikacioni izveštaj:

              precision    recall  f1-score   support

jumping_jack       0.87      0.81      0.84        16
     pull_up       0.64      0.93      0.76        15
     push_up       0.94      1.00      0.97        15
       situp       0.83      0.83      0.83        12
       squat       0.67      0.20      0.31        10

    accuracy                           0.79        68
   macro avg       0.79      0.76      0.74        68
weighted avg       0.80      0.79      0.77        68



In [23]:
def predict_sequence(seq_2d):
    """
    seq_2d: np.array oblika [num_frames, num_features]
    vraća: (naziv_klase, verovatnoća)
    """
    seq_list = [seq_2d.astype("float32")]

    seq_padded = pad_sequences(
        seq_list,
        maxlen=max_len,
        dtype="float32",
        padding="post",
        truncating="post",
        value=0.0
    )

    probs = model.predict(seq_padded)[0]
    class_idx = int(np.argmax(probs))
    class_name = le.inverse_transform([class_idx])[0]
    confidence = float(probs[class_idx])

    return class_name, confidence


In [24]:
test_seq = X_test_list[0]
true_label = y_test[0]

pred_label, conf = predict_sequence(test_seq)

print("STVARNA klasa:", true_label)
print("PREDIKCIJA:", pred_label, f"({conf*100:.1f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
STVARNA klasa: push_up
PREDIKCIJA: push_up (35.0%)


In [25]:
import pickle, json, os

model.save("exercise_lstm_angles.keras")

with open("label_encoder.pkl", "wb") as f:
    pickle.dump(le, f)


with open("feature_cols.json", "w") as f:
    json.dump(feature_cols, f)


config = {"max_len": int(max_len)}
with open("config.json", "w") as f:
    json.dump(config, f)

os.listdir()


['best_lstm_angles.keras',
 'best_lstm_model.keras',
 'config.json',
 'data',
 'exercise_lstm_angles.keras',
 'exercise_lstm_final.keras',
 'feature_cols.json',
 'label_encoder.pkl',
 'model_training.ipynb',
 'notebooks',
 'pose_capture.ipynb',
 'pose_capture.py',
 'src',
 'venv',
 'videos']